## Scenario: Token Issuers can maintain multiple Token Status Lists

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import random

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /)

In [3]:
health_endpoint = f"{BASE_URL}/"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/
Health check successful! Server is running.


### Two Issuers register on the Status List Server by publish credentials

In [4]:
gondwana_digital_pole = fixtures.get_gondwana_digital_pole_issuer()
scott_holdings = fixtures.get_scott_holdings_issuer()

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    print("Proceeding with Issuer:", issuer_data.get('label'))
    
    credentials_endpoint = f"{BASE_URL}/credentials"
    print(f"Testing POST: {credentials_endpoint}")
    
    payload = {
      "issuer": issuer_data.get('label'),
      "public_key": issuer_data.get('keypair').get('public_key'),
      "alg": issuer_data.get('keypair').get('alg')
    }
    
    response = requests.post(credentials_endpoint, json=payload)
    tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
    print("Credentials published successfully!\n")

Proceeding with Issuer: gondwana-digital-pole-6l2rg
Testing POST: http://localhost:8000/credentials
Credentials published successfully!

Proceeding with Issuer: scott-holdings-0jd1j
Testing POST: http://localhost:8000/credentials
Credentials published successfully!



### They publish multiple status lists

In [5]:
# Since status lists are so far not linked to issuers, just assume they are.
# Below, each issuer publishes three lists.

status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    for _ in range(3): # each issuer publishes three lists
        status_list_id = str(uuid.uuid4())
        print(f"{issuer_data.get('label')} publishes status list {status_list_id}")
        
        response = requests.post(status_publish_endpoint, json={
            "list_id": status_list_id,
            "bits": random.choice([1, 2, 4, 8]),
            "updates": [{"index": i, "status": "VALID"} for i in range(random.randint(1, 5))]
        })
        
        tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")

Testing POST: http://localhost:8000/statuslists/publish
gondwana-digital-pole-6l2rg publishes status list b76cdc2b-ba85-4761-920f-730cc5e60e1f
gondwana-digital-pole-6l2rg publishes status list a210da97-b37e-4c28-a9f1-6057d68f5d25
gondwana-digital-pole-6l2rg publishes status list 31bf26d6-8cd3-414e-b442-56bc83218740
scott-holdings-0jd1j publishes status list 2a433d14-f36b-4da6-8bde-66933704a9ba
scott-holdings-0jd1j publishes status list 8900de6a-869c-47a4-a8b3-8cbe7c22c31a
scott-holdings-0jd1j publishes status list 97a64f55-2b36-4063-b763-ac9c51635422
